In [ ]:
# IMPORT ALL YOUR LIBRARIES
!pip install transformers
!pip install datasets
!pip install accelerate >=0.21.0
!pip install transformers[torch]

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from datasets import Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/sun.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/som.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/translate/deu-to-eng.csv"
# !wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/deu_c.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/translate/rus-to-eng.csv"

--2024-11-29 10:35:22--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/translate/deu-to-eng.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 609547 (595K) [text/plain]
Saving to: ‘/kaggle/working/deu-to-eng.csv’

deu-to-eng.csv      100%[===================>] 595.26K  --.-KB/s    in 0.007s  

2024-11-29 10:35:23 (80.2 MB/s) - ‘/kaggle/working/deu-to-eng.csv’ saved [609547/609547]

--2024-11-29 10:35:23--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/translate/rus-to-eng.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (ra

In [ ]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/deu.csv"

--2024-11-29 10:46:59--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/train/deu.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 680115 (664K) [text/plain]
Saving to: ‘/kaggle/working/deu.csv’

deu.csv             100%[===================>] 664.17K  --.-KB/s    in 0.005s  

2024-11-29 10:47:00 (126 MB/s) - ‘/kaggle/working/deu.csv’ saved [680115/680115]



In [ ]:
ger = pd.read_csv('/kaggle/working/deu.csv')
ger

id  \
0     deu_train_track_a_00001   
1     deu_train_track_a_00002   
2     deu_train_track_a_00003   
3     deu_train_track_a_00004   
4     deu_train_track_a_00005   
...                       ...   
2598  deu_train_track_a_02599   
2599  deu_train_track_a_02600   
2600  deu_train_track_a_02601   
2601  deu_train_track_a_02602   
2602  deu_train_track_a_02603   

                                                   text  Anger  Disgust  Fear  \
0     Nein nein nix da, fuck den schön weiter ab bis...      1        0     0   
1     Vor 100 Jahren ging man auf die Strasse weil m...      1        1     0   
2                   Wann endlich Gabby Epstein heiraten      0        0     0   
3     Wegen dem Song 1001 Nacht, dem einen Phantomto...      0        0     0   
4     Dann sollten wir versuchen, sie zu erreichen. ...      0        0     0   
...                                                 ...    ...      ...   ...   
2598  Das nenne ich guten Geschmack, mein Geologenhe...      0        0     0   
2599  Zeigt mir irgendwie, dass es auch bei den Nörg...      0        0     0   
2600  1.) Ist es möglich dein Tätigkeitsbereich etwa...      0        0     0   
2601    Egal nicht alles muss man unter Kontrolle haben      0        0     0   
2602  Klar, das ist der Grund. Auf keinen Fall die P...      1        1     0   

      Joy  Sadness  Surprise  
0       0        0         0  
1       0        0         0  
2       0        0         0  
3       1        0         0  
4       0        0         0  
...   ...      ...       ...  
2598    1        0         0  
2599    1        0         0  
2600    0        0         0  
2601    0        0         0  
2602    0        0         0  

[2603 rows x 8 columns]

In [ ]:
df = pd.read_csv('/kaggle/working/deu-to-eng.csv')
df.head()

id                                               text  \
0  deu_train_track_a_00001  No, no, no, no, no, no, no, no, no, no, no, no...   
1  deu_train_track_a_00002  100 years ago you went out on the street becau...   
2  deu_train_track_a_00003            When Gabby Epstein finally gets married   
3  deu_train_track_a_00004  Because of the song 1001 night, a phantom gate...   
4  deu_train_track_a_00005  Then we should try to reach them. On Twitter a...   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0      1        0     0    0        0         0  
1      1        1     0    0        0         0  
2      0        0     0    0        0         0  
3      0        0     0    1        0         0  
4      0        0     0    0        0         0

In [ ]:
df

id  \
0     deu_train_track_a_00001   
1     deu_train_track_a_00002   
2     deu_train_track_a_00003   
3     deu_train_track_a_00004   
4     deu_train_track_a_00005   
...                       ...   
2598  deu_train_track_a_02599   
2599  deu_train_track_a_02600   
2600  deu_train_track_a_02601   
2601  deu_train_track_a_02602   
2602  deu_train_track_a_02603   

                                                   text  Anger  Disgust  Fear  \
0     No, no, no, no, no, no, no, no, no, no, no, no...      1        0     0   
1     100 years ago you went out on the street becau...      1        1     0   
2               When Gabby Epstein finally gets married      0        0     0   
3     Because of the song 1001 night, a phantom gate...      0        0     0   
4     Then we should try to reach them. On Twitter a...      0        0     0   
...                                                 ...    ...      ...   ...   
2598  That's what I call good taste, my geologist's ...      0        0     0   
2599  Shows me somehow that there are also with the ...      0        0     0   
2600  1.) Is it possible to narrow down your area of...      0        0     0   
2601  It doesn't matter that you have to have everyt...      0        0     0   
2602  Of course, that's the reason. No way the polit...      1        1     0   

      Joy  Sadness  Surprise  
0       0        0         0  
1       0        0         0  
2       0        0         0  
3       1        0         0  
4       0        0         0  
...   ...      ...       ...  
2598    1        0         0  
2599    1        0         0  
2600    0        0         0  
2601    0        0         0  
2602    0        0         0  

[2603 rows x 8 columns]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2603 entries, 0 to 2602
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        2603 non-null   object
 1   text      2603 non-null   object
 2   Anger     2603 non-null   int64 
 3   Disgust   2603 non-null   int64 
 4   Fear      2603 non-null   int64 
 5   Joy       2603 non-null   int64 
 6   Sadness   2603 non-null   int64 
 7   Surprise  2603 non-null   int64 
dtypes: int64(6), object(2)
memory usage: 162.8+ KB


In [ ]:
df_test = pd.read_csv('/kaggle/working/rus-to-eng.csv')
df_test.head()

id                                               text  \
0  rus_dev_track_c_00001  They're gonna fucking give it to me like they ...   
1  rus_dev_track_c_00002  No one knows the strength of his ability until...   
2  rus_dev_track_c_00003                         Wow! () 62 the first time.   
3  rus_dev_track_c_00004  I can only sit on Twitter and I want to talk t...   
4  rus_dev_track_c_00005                                    Yeah, I did it!   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    NaN      NaN   NaN  NaN      NaN       NaN  
4    NaN      NaN   NaN  NaN      NaN       NaN

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401 entries, 0 to 400
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        401 non-null    object 
 1   text      401 non-null    object 
 2   Anger     0 non-null      float64
 3   Disgust   0 non-null      float64
 4   Fear      0 non-null      float64
 5   Joy       0 non-null      float64
 6   Sadness   0 non-null      float64
 7   Surprise  0 non-null      float64
dtypes: float64(6), object(2)
memory usage: 25.2+ KB


In [ ]:
# from bs4 import BeautifulSoup
# import re

# # # Step 2: Clean the text data
# # def clean_text(text):
# #     # Remove HTML tags
# #     text = BeautifulSoup(text, "html.parser").get_text()
# #     # Remove numbers (but keep special characters)
# #     text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
# #     # Convert to lowercase
# #     text = text.lower()
# #     # Remove extra whitespace
# #     text = ' '.join(text.split())
# #     return text

# # # Apply the clean_text function
# # df_test['text'] = df_test['text'].apply(clean_text)

# # Step 3-5: (Same as your original code)
# df_test = df_test[df_test['text'].notna() & (df_test['text'] != '')]

# # Ensure emotion columns are integers
# # emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['Anger', 'Disgust','Fear', 'Joy', 'Sadness', 'Surprise']
# df_test[emotion_columns] = df_test[emotion_columns]

# # Save the processed data back to CSV
# output_path = '/kaggle/working/cleaned_test_english.csv'
# df_test.to_csv(output_path, index=False)

# print(f"Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.")

# Display statistics and processed examples
print(f"Total samples: {len(df_test)}")
print("\nEmotion distribution in the dataset:")
print(df_test[emotion_columns].sum())
print("\nFirst few processed examples:")
print(df_test.head())


Total samples: 401

Emotion distribution in the dataset:
Anger       0.0
Disgust     0.0
Fear        0.0
Joy         0.0
Sadness     0.0
Surprise    0.0
dtype: float64

First few processed examples:
                      id                                               text  \
0  rus_dev_track_c_00001  They're gonna fucking give it to me like they ...   
1  rus_dev_track_c_00002  No one knows the strength of his ability until...   
2  rus_dev_track_c_00003                         Wow! () 62 the first time.   
3  rus_dev_track_c_00004  I can only sit on Twitter and I want to talk t...   
4  rus_dev_track_c_00005                                    Yeah, I did it!   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    NaN      NaN   NaN  NaN      NaN       NaN  
4    NaN      NaN   NaN  NaN      NaN       NaN  


In [ ]:
if 'id' in df.columns:
    df = df.drop(['id'], axis=1)
from bs4 import BeautifulSoup
import re

# # Step 2: Clean the text data
# def clean_text(text):
#     # Remove HTML tags
#     text = BeautifulSoup(text, "html.parser").get_text()
#     # Remove numbers (but keep special characters)
#     text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
#     # Convert to lowercase
#     text = text.lower()
#     # Remove extra whitespace
#     text = ' '.join(text.split())
#     return text

# # df['text'] = df['text'].apply(clean_text)

# Step 3: Remove rows with empty text
df = df[df['text'].notna() & (df['text'] != '')]

# Step 4: Ensure emotion columns are integers
emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# emotion_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
df[emotion_columns] = df[emotion_columns].astype(int)

# Step 5: Save the processed data back to CSV
output_path = '/kaggle/working/cleaned-deu-to-eng.csv'
df.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_som.csv' has been created in the /kaggle/working directory.")

# Optional: Display some statistics about the data
print(f"Total samples: {len(df)}")
print("\nEmotion distribution in the dataset:")
print(df[emotion_columns].sum())

# Optional: Display a few processed examples
print("\nFirst few processed examples:")
print(df.head())

Data preprocessing completed. File 'cleaned_som.csv' has been created in the /kaggle/working directory.
Total samples: 2603

Emotion distribution in the dataset:
Anger       768
Disgust     832
Fear        239
Joy         541
Sadness     516
Surprise    159
dtype: int64

First few processed examples:
                                                text  Anger  Disgust  Fear  \
0  No, no, no, no, no, no, no, no, no, no, no, no...      1        0     0   
1  100 years ago you went out on the street becau...      1        1     0   
2            When Gabby Epstein finally gets married      0        0     0   
3  Because of the song 1001 night, a phantom gate...      0        0     0   
4  Then we should try to reach them. On Twitter a...      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        0         0  
2    0        0         0  
3    1        0         0  
4    0        0         0  


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import f1_score
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
MODEL_NAME = "xlm-roberta-base"  # Change to "bert-base-multilingual-cased" for mBERT
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=6)  # 6 emotions


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
train_dataset = Dataset.from_pandas(df)

train_dataset

dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 2082
    })
    test: Dataset({
        features: ['text', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 521
    })
})


In [ ]:
# Convert train_dataset and val_dataset back to Pandas DataFrame
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()

# Save to CSV
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)

print("CSV files have been saved.")


CSV files have been saved.


In [ ]:
train_df.head()

text  Anger  Disgust  Fear  \
0  As long as the war is not won, climate bloatin...      1        1     0   
1  President Zelensky is speaking to ex-US Presid...      1        1     0   
2  Feelings are never ridiculous, I have to disap...      0        0     0   
3  ch hope the federal government shows solidarit...      1        0     0   
4  The fact that a prominent feminist is hopeless...      0        1     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        0         0  
2    0        0         0  
3    0        0         0  
4    0        1         0

In [ ]:
val_df.head()

text  Anger  Disgust  Fear  \
0  It is now part of the good tone to equate the ...      1        1     0   
1  As described in the title: We are parents of t...      0        0     0   
2  I used to have something with one and I read i...      0        0     0   
3  The Free Voters had also voted as a majority f...      0        1     0   
4  I got a blow from my girlfriend at the time. H...      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        1         0  
2    1        0         0  
3    0        0         1  
4    1        0         1

In [ ]:
# #print the values in train_dataset

# print(train_dataset[0])

In [ ]:
# #for each entry in train_dataset, merge Anger, Fear, Joy, Sadness, Surprise

# train_dataset = train_dataset.map(lambda x: {'labels': str(x['Anger']) + ' + '  + str(x['Fear']) + ' + '  +  str(x['Joy']) + ' + '  + str(x['Sadness']) +  ' + ' + str(x['Surprise'])})

# #drop Anger Fear Joy Sadness and Surprise

# train_dataset = train_dataset.remove_columns(['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])

# #print the labels coloumn to check

# print(train_dataset)

# #print labels
# print(train_dataset[0])

In [ ]:
# #do the same for validation dataset

# val_dataset = val_dataset.map(lambda x: {'labels': str(x['Anger']) + ' + '  + str(x['Fear']) + ' + '  +  str(x['Joy']) + ' + '  + str(x['Sadness']) +  ' + ' + str(x['Surprise'])})

# val_dataset = val_dataset.remove_columns(['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'])

# print(val_dataset)

# #print labels
# print(val_dataset[0])

In [ ]:
cols = train_df.columns[1:]
cols

Index(['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise'], dtype='object')

In [ ]:
training_labels = train_df[cols]
training_labels = training_labels.to_numpy()
training_labels = training_labels.astype(np.float32)

train_sentences = train_df.text.values.tolist()

validation_labels = val_df[cols]
validation_labels = validation_labels.to_numpy()
validation_labels = validation_labels.astype(np.float32)

validation_sentences = val_df.text.values.tolist()

In [ ]:
# training_sentence_encoding = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
# validation_sentence_encoding = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)

train_encodings = tokenizer(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(validation_sentences, padding=True, truncation=True, max_length=512)

In [ ]:
# #defining a custom dataset class for the training and validation datasets to be used in the trainer

# class ArgumentDataset(Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# #creating the training and validation datasets
# train_dataset_new = ArgumentDataset(training_sentence_encoding, training_labels)
# validation_dataset = ArgumentDataset(validation_sentence_encoding, validation_labels)

# #train_dataset[0]

In [ ]:
from datasets import Dataset
import torch
import numpy as np

train_dataset_new = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': training_labels
})

validation_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': validation_labels
})

In [ ]:
# print(type(train_dataset_new))

In [ ]:
# print(train_dataset_new[0])

In [ ]:
# source = train_dataset['text']
# target = train_dataset['labels']

# max_length = tokenizer.model_max_length

# tokenizer.model_max_length = max_length

In [ ]:
# def tokenize_function(examples):
#     tokenized_inputs = tokenizer(
#         examples["text"],
#         padding="max_length",  # Pad to the max length
#         truncation=True,       # Truncate to avoid exceeding model limits
#         max_length=max_length
#     )

#     tokenized_labels = tokenizer(
#         examples["labels"],
#         padding="max_length",  # Pad to the max length
#         truncation=True,       # Truncate labels as well
#         max_length=max_length
#     )

#     # Ensure that tokenized_labels are added as 'labels'
#     tokenized_inputs["labels"] = examples["labels"]

#     return tokenized_inputs



# # Tokenize the training and validation datasets
# tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
# val_dataset = val_dataset.map(tokenize_function, batched=True)

# # Format the dataset for training
# tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
# val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# print(tokenized_dataset[0])

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

epochs = 4
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(df) * epochs)

training_arguments = TrainingArguments(
    output_dir="./outputs",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import wandb

# Initialize wandb with mode disabled
wandb.init(mode="disabled")

In [ ]:
#defining the metrics to be calculated during training
def multi_label_metrics(predictions, labels,):
    sigmoid = torch.nn.Sigmoid()
    predictions = sigmoid(torch.tensor(predictions))
    y_predictions = np.zeros(predictions.shape)
    y_true = labels
    y_predictions[np.where(predictions >= 0.4)] = 1
    accuracy = accuracy_score(y_true, y_predictions)
    f1 = f1_score(y_true, y_predictions, average='micro')
    auc = roc_auc_score(y_true, y_predictions, average='micro')

    metrics = {
        'accuracy': accuracy,
        'f1': f1,
        'auc': auc
    }

    return metrics

#defining the function to compute the metrics for each batch of data
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(preds, p.label_ids)

    return result

#defining the trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset_new,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


TrainOutput(global_step=524, training_loss=0.3402510572025794, metrics={'train_runtime': 637.6634, 'train_samples_per_second': 13.06, 'train_steps_per_second': 0.822, 'total_flos': 791766600360480.0, 'train_loss': 0.3402510572025794, 'epoch': 4.0})

In [ ]:
#Saving the model
model.save_pretrained("/content/")

In [ ]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/dev/rus_c.csv"

--2024-11-29 08:16:57--  https://raw.githubusercontent.com/MuhammadAreebKazmi2/Deep-Learning/refs/heads/main/Project/Dataset/TaskC/dev/rus_c.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56796 (55K) [text/plain]
Saving to: ‘/kaggle/working/rus_c.csv’

rus_c.csv           100%[===================>]  55.46K  --.-KB/s    in 0.004s  

2024-11-29 08:16:57 (14.0 MB/s) - ‘/kaggle/working/rus_c.csv’ saved [56796/56796]



In [ ]:
rus = pd.read_csv('/kaggle/working/rus_c.csv')
rus.head()

id                                               text  \
0  rus_dev_track_c_00001                           как всегда хуйни зададут   
1  rus_dev_track_c_00002  Никто не знает силы своих способностей, пока о...   
2  rus_dev_track_c_00003                           Ого!!) 62 с первого раза   
3  rus_dev_track_c_00004  Блин! Могу сидеть только в твиттере, а так хоч...   
4  rus_dev_track_c_00005           Даа! Я это сделала!! Мама уезжает!))))))   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0    NaN      NaN   NaN  NaN      NaN       NaN  
1    NaN      NaN   NaN  NaN      NaN       NaN  
2    NaN      NaN   NaN  NaN      NaN       NaN  
3    NaN      NaN   NaN  NaN      NaN       NaN  
4    NaN      NaN   NaN  NaN      NaN       NaN

In [ ]:
#checking the training history of the model
trainer.state.log_history

[{'eval_loss': 0.4131099581718445,
  'eval_accuracy': 0.3493282149712092,
  'eval_f1': 0.49396471680594245,
  'eval_auc': 0.6866568842295367,
  'eval_runtime': 15.4512,
  'eval_samples_per_second': 33.719,
  'eval_steps_per_second': 2.136,
  'epoch': 1.0,
  'step': 131},
 {'eval_loss': 0.39970698952674866,
  'eval_accuracy': 0.3723608445297505,
  'eval_f1': 0.5285592497868713,
  'eval_auc': 0.7152250878216917,
  'eval_runtime': 15.6005,
  'eval_samples_per_second': 33.396,
  'eval_steps_per_second': 2.115,
  'epoch': 2.0,
  'step': 262},
 {'eval_loss': 0.3972603678703308,
  'eval_accuracy': 0.3723608445297505,
  'eval_f1': 0.550210970464135,
  'eval_auc': 0.7300914556927177,
  'eval_runtime': 15.5631,
  'eval_samples_per_second': 33.477,
  'eval_steps_per_second': 2.12,
  'epoch': 3.0,
  'step': 393},
 {'loss': 0.3444,
  'grad_norm': 4.6013503074646,
  'learning_rate': 2.2900763358778625e-06,
  'epoch': 3.816793893129771,
  'step': 500},
 {'eval_loss': 0.38009098172187805,
  'eval_accu

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the model and tokenizer
# model = BertForSequenceClassification.from_pretrained("/content/")
# tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForSequenceClassification.from_pretrained("/content/", num_labels=6)
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")


# MODEL_NAME = "xlm-roberta-base"  # Change to "bert-base-multilingual-cased" for mBERT
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=6)  # 6 emotions

# Make sure to set the model to evaluation mode
model.eval()


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [ ]:
# Test data (assuming you have a DataFrame with a column 'text')
test_sentences = df_test['text'].tolist()

# Tokenize the test data
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
# Move the encodings to the same device as the model (GPU if available, otherwise CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Pass the test data through the model to get predictions
with torch.no_grad():
    input_ids = test_encodings['input_ids'].to(device)
    attention_mask = test_encodings['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)

# Apply sigmoid to get probabilities
sigmoid = torch.nn.Sigmoid()
predictions = sigmoid(outputs.logits)

# Convert probabilities to binary predictions (assuming a threshold of 0.5)
predicted_labels = (predictions > 0.5).int()

# Convert predictions to a more readable format (back to CPU if necessary)
predicted_labels = predicted_labels.cpu().numpy()


In [ ]:
# Assuming your labels are in the order ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
label_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Convert predictions to a DataFrame for easy viewing
predicted_df = pd.DataFrame(predicted_labels, columns=label_columns)

# Combine with the original test sentences
result_df = pd.concat([df_test['text'], predicted_df], axis=1)

# Display the result
print(result_df.head())


                                                text  Anger  Disgust  Fear  \
0  They're gonna fucking give it to me like they ...      0        1     0   
1  No one knows the strength of his ability until...      0        0     0   
2                         Wow! () 62 the first time.      0        0     0   
3  I can only sit on Twitter and I want to talk t...      0        0     0   
4                                    Yeah, I did it!      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         0  
1    0        0         0  
2    1        0         0  
3    1        0         0  
4    1        0         0  


In [ ]:
# Save predictions to CSV
result_df.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'.")


Predictions saved to 'predictions.csv'.


In [ ]:
# Load the Russian and English files
russian_file_path = "/kaggle/working/rus_c.csv" # Replace with the actual path
english_file_path = "predictions.csv"  # Replace with the actual path

russian_df = pd.read_csv(russian_file_path)
english_df = pd.read_csv(english_file_path)

# Reset index of english_df to match id of russian_df
english_df['id'] = russian_df['id']

# Merge the dataframes on the 'id' column to copy the emotion labels
merged_df = russian_df.merge(
    english_df[['id', 'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']],
    on='id',
    how='left'
)

# Rename *_y columns to proper names and drop *_x columns if they exist
emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
for emotion in emotion_columns:
    if f"{emotion}_x" in merged_df.columns:  # Drop the *_x columns
        merged_df.drop(columns=[f"{emotion}_x"], inplace=True, errors='ignore')
    if f"{emotion}_y" in merged_df.columns:  # Rename *_y to the original name
        merged_df.rename(columns={f"{emotion}_y": emotion}, inplace=True)


# Save the merged dataframe back to a CSV
output_file_path = "rus-predictions.csv"  # Replace with the desired output path
merged_df.to_csv(output_file_path, index=False)

print("Merged data saved to:", output_file_path)


Merged data saved to: rus-predictions.csv


In [ ]:
csv = pd.read_csv('rus-predictions.csv')
csv.head()

id                                               text  \
0  rus_dev_track_c_00001                           как всегда хуйни зададут   
1  rus_dev_track_c_00002  Никто не знает силы своих способностей, пока о...   
2  rus_dev_track_c_00003                           Ого!!) 62 с первого раза   
3  rus_dev_track_c_00004  Блин! Могу сидеть только в твиттере, а так хоч...   
4  rus_dev_track_c_00005           Даа! Я это сделала!! Мама уезжает!))))))   

   Anger  Disgust  Fear  Joy  Sadness  Surprise  
0      0        1     0    0        0         0  
1      0        0     0    0        0         0  
2      0        0     0    1        0         0  
3      0        0     0    1        0         0  
4      0        0     0    1        0         0

In [ ]:
import pandas as pd

# Step 1: Load the predictions CSV file
df = pd.read_csv('rus-predictions.csv')

# Step 2: Replace 'text' column with an 'id' column
df['id'] = [f"rus_dev_track_c_{str(i+1).zfill(5)}" for i in range(len(df))]

# Step 3: Drop the original 'text' column
df = df.drop(columns=['text'])

# Step 4: Reorder columns so 'id' comes first, followed by the emotions in the correct order
df = df[['id',  'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Step 5: Save the updated CSV file for submission
df.to_csv('final_submission.csv', index=False)

print("Submission file 'final_submission.csv' has been created.")


Submission file 'final_submission.csv' has been created.


In [ ]:
final = pd.read_csv('final_submission.csv')
final.head()

id  Anger  Disgust  Fear  Joy  Sadness  Surprise
0  rus_dev_track_c_00001      0        1     0    0        0         0
1  rus_dev_track_c_00002      0        0     0    0        0         0
2  rus_dev_track_c_00003      0        0     0    1        0         0
3  rus_dev_track_c_00004      0        0     0    1        0         0
4  rus_dev_track_c_00005      0        0     0    1        0         0

In [ ]:
# import numpy as np
# import torch
# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# # Defining the metrics to be calculated during training
# def multi_class_multi_label_metrics(predictions, labels):
#     sigmoid = torch.nn.Sigmoid()
#     predictions = sigmoid(torch.tensor(predictions))

#     # Ensure predictions are rounded to 0 and 1
#     predictions_rounded = np.round(predictions.detach().numpy())  # Convert to numpy and round

#     # Use labels directly if they are already numpy arrays
#     y_true = labels  # Assuming labels are in numpy format

#     # Check shapes for debugging
#     print("Rounded Predictions shape:", predictions_rounded.shape)
#     print("True Labels shape:", y_true.shape)

#     # Calculate metrics
#     accuracy = accuracy_score(y_true, predictions_rounded)
#     f1 = f1_score(y_true, predictions_rounded, average='micro')
#     auc = roc_auc_score(y_true, predictions_rounded, average='micro')

#     metrics = {
#         'accuracy': accuracy,
#         'f1': f1,
#         'auc': auc
#     }

#     return metrics

# # Defining the function to compute the metrics for each batch of data
# def compute_metrics(p):
#     preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
#     result = multi_class_multi_label_metrics(preds, p.label_ids)

#     return result
